# Imports

In [ ]:
# stdlib
import shutil
from datetime import datetime
import os

# 3rd party
import numpy as np
from IPython.display import display

# local
import kelp_compute as kc
import kelp_analyze as ka

# Test Functions

In [ ]:
def create_phony_kelp(data_path):
    ns, nz, na = np.random.randint(10, 20, size=3)
    # Make sure na is even
    if na % 2 == 1:
        na -= 1
        
    absorptance_kelp, a_water, b = np.random.rand(3)
    num_dens = np.random.rand()
    fs, fr, ft = np.random.rand(3)
    kelp_dist = 'top-heavy'
    max_length, length_std = np.random.rand(2)
    zmax, rope_spacing = 10 * np.random.rand(2)
    date = datetime.now().ctime()
    compute_time = 100 * np.random.rand()
    git_commit = kc.get_git_commit_hash()
    num_cores = np.random.randint(64)
    num_scatters = np.random.randint(20)
    fd_flag = np.random.randint(2)
    lis_opts = '-i gmres -tol 1e-5'
    lis_iter = np.random.randint(1000)
    lis_time = 100 * np.random.rand()
    lis_resid = 1e-5 * np.random.rand()
    
    return {
        'absorptance_kelp': absorptance_kelp,
        'a_water': a_water,
        'b': b,
        'ns': ns,
        'nz': nz,
        'na': na,
        'num_dens': num_dens,
        'kelp_dist': kelp_dist,
        'fs': fs,
        'fr': fr,
        'ft': ft,
        'max_length': max_length,
        'length_std': length_std,
        'zmax': zmax,
        'rope_spacing': rope_spacing,
        'date': date,
        'compute_time': compute_time,
        'git_commit': git_commit,
        'num_cores': num_cores,
        'num_scatters': num_scatters,
        'fd_flag': fd_flag,
        'lis_opts': lis_opts,
        'lis_iter': lis_iter,
        'lis_time': lis_time,
        'lis_resid': lis_resid,
        'data_path': data_path
    }

In [ ]:
def create_phony_results(ns, nz, na):
    results = {
        # Dimension values
        # These are grid-starts, not grid-centers, but it's phony anyhow.
        'x': np.linspace(0, 1, ns),
        'y': np.linspace(0, 1, ns),
        'z': np.linspace(0, 1, nz),
        'theta': np.linspace(0, 2*np.pi, na),
        'phi': np.linspace(0, np.pi, na),
        
        # Results
        'rad': np.random.rand(ns, ns, nz, na, na),
        'irrad': np.random.rand(ns, ns, nz),
        'p_kelp': np.random.rand(ns, ns, nz),
        'avg_irrad': np.random.rand(nz),
        'perc_irrad': np.random.rand(nz)
    }
    
    return results

In [ ]:
def run_phony(conn, run_name, data_dir):
    data_path = kc.get_random_unused_filename(dir=data_dir, suffix='.nc')
    
    metadata = create_phony_kelp(data_path)
    ns = metadata['ns']
    nz = metadata['nz']
    na = metadata['na']
    results = create_phony_results(ns, nz, na)
    
    kc.create_nc(**metadata, **results)
    kc.insert_run(conn, db_name=run_name, **metadata)

def run_phony_set(run_name, prefix='.', num_entries=10):
    base_dir = os.path.join(prefix, run_name)
    data_dir = os.path.join(base_dir, 'data')
    conn = kc.create_table(run_name, prefix)
    for i in range(num_entries):
        run_phony(conn, run_name, data_dir)
        
    return conn

# Test

In [ ]:
run_name = 'test1'
prefix='.'
base_dir = os.path.join(prefix, run_name)

# Remove dir if it exists
shutil.rmtree(base_dir, ignore_errors=True)

# Run set
conn = run_phony_set(run_name, prefix=prefix, num_entries=10)
display(ka.table_to_df(conn, run_name))

# Look at results

In [ ]:
ds, *_ = ka.query_results(conn, 'test1', kelp_dist='top-heavy', id=1)

print(ds['z'])
print(ds['z'][:])

print(ds['perc_irrad'])
print(ds['perc_irrad'][:])

print(ds['compute_time'])
print(ds['compute_time'][:])

print(ds['git_commit'])
print(ds['git_commit'][:])